In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import os, glob, cv2
import numpy as np
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
train_path = '../dataset/images/'
classes = ["100", "118", "217"]

In [3]:
def load_data_cnn(train_path):
    images = []
    labels = []
    classes = [lists for lists in os.walk(train_path)][0][1]    
    for cls in classes:
        file_path = os.path.join(train_path, cls, "*g")
        for file in glob.glob(file_path):
            image = cv2.imread(file)
            image = image[288-250:288-35, 55:390]
            image = cv2.resize(image, (int(215/5), int(215/5)), cv2.INTER_LINEAR)
            label = np.zeros(len(classes))
            label[classes.index(cls)] = 1.0
            images.append(image)
            labels.append(label) 
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

dataset = load_data_cnn(train_path)

In [4]:
X_train = []
X_train.append(dataset[0][:80])
X_train.append(dataset[0][100:180])
X_train.append(dataset[0][200:280])
X_train = [j for i in X_train for j in i]
X_train = np.array(X_train)

y_train = []
y_train.append(dataset[1][:80])
y_train.append(dataset[1][100:180])
y_train.append(dataset[1][200:280])
y_train = [j for i in y_train for j in i]
y_train = np.array(y_train)

X_test = []
X_test.append(dataset[0][80:100])
X_test.append(dataset[0][180:200])
X_test.append(dataset[0][280:300])
X_test = [j for i in X_test for j in i]
X_test = np.array(X_test)

y_test = []
y_test.append(dataset[1][80:100])
y_test.append(dataset[1][180:200])
y_test.append(dataset[1][280:300])
y_test = [j for i in y_test for j in i]
y_test = np.array(y_test)

y_test_label = []
for i in range(60):
    if i >= 40:
        y_test_label.append(2)
    elif i >= 20:
        y_test_label.append(1)
    else:
        y_test_label.append(0)
        

In [5]:
input_shape = (int(215/5), int(215/5), 3)

In [6]:
cnn_model = Sequential()

cnn_model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=input_shape))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
cnn_model.add(Conv2D(32, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
cnn_model.add(Conv2D(32, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Flatten())
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(3, activation='softmax'))

In [7]:
cnn_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
classifier = cnn_model.fit(X_train, y_train, validation_split=0.1, epochs=60, batch_size=9)

Train on 216 samples, validate on 24 samples
Epoch 1/60
216/216 [==============================] - 3s 16ms/step - loss: 11.5876 - acc: 0.2685 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 2/60
216/216 [==============================] - 2s 10ms/step - loss: 11.3825 - acc: 0.2778 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 3/60
216/216 [==============================] - 2s 10ms/step - loss: 10.9433 - acc: 0.3056 - val_loss: 1.0791e-05 - val_acc: 1.0000
Epoch 4/60
216/216 [==============================] - 2s 10ms/step - loss: 11.1846 - acc: 0.2731 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 5/60
216/216 [==============================] - 2s 10ms/step - loss: 10.0209 - acc: 0.3426 - val_loss: 2.9802e-06 - val_acc: 1.0000
Epoch 6/60
216/216 [==============================] - 2s 10ms/step - loss: 9.8253 - acc: 0.3287 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 7/60
216/216 [==============================] - 2s 10ms/step - loss: 9.5059 - acc: 0.3565 - val_loss: 16.1181 - val_acc: 

In [9]:
loss, accuracy = cnn_model.evaluate(X_test, y_test)
print("Loss = "+str(loss))
print("Accuracy = "+str(accuracy))

60/60 [==============================] - 0s 3ms/step
Loss = 0.003644379631926616
Accuracy = 1.0


In [10]:
predictions = cnn_model.predict_classes(X_test, batch_size=5)

In [11]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [12]:
cn = confusion_matrix(y_test_label, predictions)

In [13]:
print ("Confusion Matrix")
cn

Confusion Matrix


array([[20,  0,  0],
       [ 0, 20,  0],
       [ 0,  0, 20]])

In [14]:
cnn_model.save("../models/cnn_3labels_2.h5")